<a href="https://colab.research.google.com/github/SazidAF/MachineLearningProjects/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk # Tokenizing
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec

In [4]:
# df = pd.read_csv('/content/drive/MyDrive/movie.csv')
df = pd.read_csv('/content/movie.csv')

In [5]:
df.shape

(40000, 2)

In [6]:
df.head

<bound method NDFrame.head of                                                     text  label
0      I grew up (b. 1965) watching and loving the Th...      0
1      When I put this movie in my DVD player, and sa...      0
2      Why do people who do not know what a particula...      0
3      Even though I have great interest in Biblical ...      0
4      Im a die hard Dads Army fan and nothing will e...      1
...                                                  ...    ...
39995  "Western Union" is something of a forgotten cl...      1
39996  This movie is an incredible piece of work. It ...      1
39997  My wife and I watched this movie because we pl...      0
39998  When I first watched Flatliners, I was amazed....      1
39999  Why would this film be so good, but only gross...      1

[40000 rows x 2 columns]>

In [7]:
df.isnull().sum() # Count the number of missing or null values in each column

text     0
label    0
dtype: int64

In [8]:
df['label'].value_counts() # 1 for positive, 0 for negative

0    20019
1    19981
Name: label, dtype: int64

In [9]:
# we are using dropping here to deal with missing values Because the dataset is large and almost no values are missing (none)
for column in df.columns:
    df = df.dropna(subset=[column]) # This drops the rows that does not have a label that is 0 or 1

In [10]:
df['text']

0        I grew up (b. 1965) watching and loving the Th...
1        When I put this movie in my DVD player, and sa...
2        Why do people who do not know what a particula...
3        Even though I have great interest in Biblical ...
4        Im a die hard Dads Army fan and nothing will e...
                               ...                        
39995    "Western Union" is something of a forgotten cl...
39996    This movie is an incredible piece of work. It ...
39997    My wife and I watched this movie because we pl...
39998    When I first watched Flatliners, I was amazed....
39999    Why would this film be so good, but only gross...
Name: text, Length: 40000, dtype: object

In [11]:
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = set(stopwords.words('english'))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
df = df[:1000]

In [14]:
text = df['text'].to_list()

In [15]:
text[0]

'I grew up (b. 1965) watching and loving the Thunderbirds. All my mates at school watched. We played "Thunderbirds" before school, during lunch and after school. We all wanted to be Virgil or Scott. No one wanted to be Alan. Counting down from 5 became an art form. I took my children to see the movie hoping they would get a glimpse of what I loved as a child. How bitterly disappointing. The only high point was the snappy theme tune. Not that it could compare with the original score of the Thunderbirds. Thankfully early Saturday mornings one television channel still plays reruns of the series Gerry Anderson and his wife created. Jonatha Frakes should hand in his directors chair, his version was completely hopeless. A waste of film. Utter rubbish. A CGI remake may be acceptable but replacing marionettes with Homo sapiens subsp. sapiens was a huge error of judgment.'

In [16]:
def prep(reviewText):
  tokens = nltk.word_tokenize(reviewText)
  tokens = [word.lower() for word in tokens if word.isalpha()]
  words = tokens[:]
  for word in tokens:
    if word in english_stopwords:
      words.remove(word)
  tokens = [lemmatizer.lemmatize(word) for word in words]
  
  return tokens

In [17]:
tokenized_text = text[:]
for i, el in enumerate(text):
  tokenized_text[i] = prep(el)

In [18]:
tokenized_text[0]

['grew',
 'b',
 'watching',
 'loving',
 'thunderbird',
 'mate',
 'school',
 'watched',
 'played',
 'thunderbird',
 'school',
 'lunch',
 'school',
 'wanted',
 'virgil',
 'scott',
 'one',
 'wanted',
 'alan',
 'counting',
 'became',
 'art',
 'form',
 'took',
 'child',
 'see',
 'movie',
 'hoping',
 'would',
 'get',
 'glimpse',
 'loved',
 'child',
 'bitterly',
 'disappointing',
 'high',
 'point',
 'snappy',
 'theme',
 'tune',
 'could',
 'compare',
 'original',
 'score',
 'thunderbird',
 'thankfully',
 'early',
 'saturday',
 'morning',
 'one',
 'television',
 'channel',
 'still',
 'play',
 'rerun',
 'series',
 'gerry',
 'anderson',
 'wife',
 'created',
 'jonatha',
 'frakes',
 'hand',
 'director',
 'chair',
 'version',
 'completely',
 'hopeless',
 'waste',
 'film',
 'utter',
 'rubbish',
 'cgi',
 'remake',
 'may',
 'acceptable',
 'replacing',
 'marionette',
 'homo',
 'sapiens',
 'subsp',
 'sapiens',
 'huge',
 'error',
 'judgment']

In [19]:
max_len = max(len(seq) for seq in tokenized_text)
max_len

558

In [20]:
w2vModel = Word2Vec(min_count=1, size=max_len, window=5)
w2vModel.build_vocab(tokenized_text)

In [21]:
print(list(w2vModel.wv.vocab.items())[:20])
print(type(w2vModel.wv.vocab))

[('grew', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45190>), ('b', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45490>), ('watching', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45220>), ('loving', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45e50>), ('thunderbird', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45f10>), ('mate', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45f70>), ('school', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ff45f40>), ('watched', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ae1f070>), ('played', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ae1f0d0>), ('lunch', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ae1f130>), ('wanted', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ae1f190>), ('virgil', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ae1f1f0>), ('scott', <gensim.models.keyedvectors.Vocab object at 0x7f3c5ae1f250>), ('one', <gensim.models.keyedvectors.Vocab object at 0

In [22]:
w2vModel.train(tokenized_text, total_examples=w2vModel.corpus_count, epochs=w2vModel.epochs)

(565649, 611835)

In [23]:
w2vModel.wv[tokenized_text[0][0]]

array([-1.84498597e-02, -4.48935805e-03, -6.29927367e-02, -1.24959648e-02,
       -4.34011519e-02,  4.09505218e-02,  7.83157162e-03,  3.85517254e-02,
       -6.65066391e-02,  1.16884178e-02,  3.63344550e-02,  1.01238501e-03,
       -2.03976035e-02,  1.22069344e-02, -8.92073289e-03,  6.06069006e-02,
        4.08843569e-02, -7.93889072e-03,  2.27746405e-02, -2.08670963e-02,
        7.54790604e-02, -2.14037709e-02, -3.85414585e-02,  2.10420694e-02,
       -9.64888651e-03, -3.58342528e-02, -2.66724955e-02, -3.28041357e-03,
       -2.16959813e-03,  9.06190928e-03,  1.97203383e-02,  3.04416521e-03,
        6.83523268e-02, -2.04340126e-02, -4.73972298e-02,  1.94646083e-02,
       -1.46795651e-02,  6.98879212e-02,  3.35324928e-03,  8.93194415e-03,
        5.58280386e-02, -2.19408311e-02,  1.60374269e-02, -2.23664697e-02,
        5.33492165e-03,  5.42208627e-02,  1.36763053e-02,  2.54918505e-02,
        3.94192040e-02, -4.43224609e-03, -3.74505892e-02,  4.50456887e-02,
       -1.62655693e-02, -

In [24]:
label = df['label'].to_list()

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
vectorized_text = np.zeros((len(tokenized_text), max_len))
for i, tokens in enumerate(tokenized_text):
    for j, token in enumerate(tokens):
        vectorized_text[i,j] = w2vModel.wv[token][0]


In [27]:
labels = np.array(label)

In [28]:
n = len(vectorized_text)
idx = np.random.permutation(n)
train_idx, val_idx, test_idx = idx[:int(0.6*n)], idx[int(0.6*n):int(0.8*n)], idx[int(0.8*n):]
X_train, X_val, X_test = vectorized_text[train_idx], vectorized_text[val_idx], vectorized_text[test_idx]
y_train, y_val, y_test = labels[train_idx], labels[val_idx], labels[test_idx]

In [29]:
input_size = max_len
hidden_size = 128
output_size = 2
num_layers = 2
learning_rate = 0.001
num_epochs = 10
batch_size = 32

In [30]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device).contiguous()
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out


In [31]:
model = RNN(input_size, hidden_size, num_layers, output_size)

In [32]:
X_train

array([[-0.07517263, -0.04115251, -0.03320409, ...,  0.        ,
         0.        ,  0.        ],
       [-0.06314987, -0.21273158, -0.17284973, ...,  0.        ,
         0.        ,  0.        ],
       [-0.16375427, -0.11278655, -0.21640123, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.0015028 , -0.03823052, -0.00563887, ...,  0.        ,
         0.        ,  0.        ],
       [-0.02269534, -0.12160052, -0.0182781 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.14374173, -0.05762868, -0.01031788, ...,  0.        ,
         0.        ,  0.        ]])

In [33]:
X_train_tensor, y_train_tensor = torch.from_numpy(X_train).float().unsqueeze(1), torch.from_numpy(y_train)

In [34]:
X_train_tensor.dim()

3

In [35]:
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [36]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [37]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [39]:
model.to(device)


RNN(
  (rnn): RNN(558, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

In [40]:
# Train model
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Epoch {}, Batch {}: Loss = {:.4f}'.format(epoch, batch_idx, loss.item()))

Epoch 0, Batch 0: Loss = 0.6970
Epoch 1, Batch 0: Loss = 0.6926
Epoch 2, Batch 0: Loss = 0.6668
Epoch 3, Batch 0: Loss = 0.6338
Epoch 4, Batch 0: Loss = 0.5957
Epoch 5, Batch 0: Loss = 0.5440
Epoch 6, Batch 0: Loss = 0.5384
Epoch 7, Batch 0: Loss = 0.6184
Epoch 8, Batch 0: Loss = 0.4945
Epoch 9, Batch 0: Loss = 0.5334
